In [1]:
import os
!pwd

/c/Users/abhis/OneDrive/Desktop/Projects/end-to-end-cdc/chicken-diseases-classification/research


In [2]:
os.chdir("c:/Users/abhis/OneDrive/Desktop/Projects/end-to-end-cdc/chicken-diseases-classification")
!pwd

/c/Users/abhis/OneDrive/Desktop/Projects/end-to-end-cdc/chicken-diseases-classification


In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int


In [ ]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json


In [25]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chicken-fecal-images",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE

        )

        return eval_config
    


In [26]:
import tensorflow as tf
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [27]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [30]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        path = Path(path)
        if not path.exists():
            raise FileNotFoundError(f"Model file not found at {path}. Please ensure the file exists.")
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
"""
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")
"""
    

'\n    def log_into_mlflow(self):\n        mlflow.set_registry_uri(self.config.mlflow_uri)\n        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme\n        \n        with mlflow.start_run():\n            mlflow.log_params(self.config.all_params)\n            mlflow.log_metrics(\n                {"loss": self.score[0], "accuracy": self.score[1]}\n            )\n            # Model registry does not work with file store\n            if tracking_url_type_store != "file":\n\n                # Register the model\n                # There are other ways to use the Model Registry, which depends on the use case,\n                # please refer to the doc for more information:\n                # https://mlflow.org/docs/latest/model-registry.html#api-workflow\n                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")\n            else:\n                mlflow.keras.log_model(self.model, "model")\n'

In [32]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    #evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-05-08 23:03:44,038: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-08 23:03:44,041: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-08 23:03:44,043: INFO: common: created directory at: artifacts]
Found 116 images belonging to 2 classes.
8/8 [==============================] - 11s 1s/step - loss: 5.0286 - accuracy: 0.5086
